In [3]:
import numpy as np
np.random.seed(42)  # for reproducibility
import matplotlib.pyplot as plt
from datetime import datetime

import bilby
from bilby.gw.utils import noise_weighted_inner_product
# dont print bilby output
bilby.core.utils.logger.setLevel("ERROR")

from gwsnr.utils import get_gw_parameters

In [4]:
Gamma = 0.5772156649015329
Pi = np.pi
MTSUN_SI = 4.925491025543576e-06

def calculate_time_to_merger(m1, m2, fmin):
    """
    Time taken from f_min to f_lso (last stable orbit). 3.5PN in fourier phase considered.

    Parameters
    ----------
    m1 : `float`
        Mass of the first body in solar masses.
    m2 : `float`
        Mass of the second body in solar masses.
    fmin : `float`
        Lower frequency cutoff.

    Returns
    -------
    chirp_time : float
        Time taken from f_min to f_lso (last stable orbit frequency).
    """

    # variables used to compute chirp time
    m = m1 + m2
    eta = m1 * m2 / m / m
    c0T = c2T = c3T = c4T = c5T = c6T = c6LogT = c7T = 0.0

    c7T = Pi * (
        14809.0 * eta * eta / 378.0 - 75703.0 * eta / 756.0 - 15419335.0 / 127008.0
    )

    c6T = (
        Gamma * 6848.0 / 105.0
        - 10052469856691.0 / 23471078400.0
        + Pi * Pi * 128.0 / 3.0
        + eta * (3147553127.0 / 3048192.0 - Pi * Pi * 451.0 / 12.0)
        - eta * eta * 15211.0 / 1728.0
        + eta * eta * eta * 25565.0 / 1296.0
        + eta * eta * eta * 25565.0 / 1296.0
        + np.log(4.0) * 6848.0 / 105.0
    )
    c6LogT = 6848.0 / 105.0

    c5T = 13.0 * Pi * eta / 3.0 - 7729.0 * Pi / 252.0

    c4T = 3058673.0 / 508032.0 + eta * (5429.0 / 504.0 + eta * 617.0 / 72.0)
    c3T = -32.0 * Pi / 5.0
    c2T = 743.0 / 252.0 + eta * 11.0 / 3.0
    c0T = 5.0 * m * MTSUN_SI / (256.0 * eta)

    # This is the PN parameter v evaluated at the lower freq. cutoff
    xT = np.power(Pi * m * MTSUN_SI * fmin, 1.0 / 3.0)
    x2T = xT * xT
    x3T = xT * x2T
    x4T = x2T * x2T
    x5T = x2T * x3T
    x6T = x3T * x3T
    x7T = x3T * x4T
    x8T = x4T * x4T

    # Computes the chirp time as tC = t(v_low)
    # tC = t(v_low) - t(v_upper) would be more
    # correct, but the difference is negligble.
    return (
        c0T
        * (
            1
            + c2T * x2T
            + c3T * x3T
            + c4T * x4T
            + c5T * x5T
            + (c6T + c6LogT * np.log(xT)) * x6T
            + c7T * x7T
        )
        / x8T
    )

## Data generation with ler

In [5]:
# from ler.gw_source_population import CBCSourceParameterDistribution

# ler = CBCSourceParameterDistribution()

# param_dict = ler.sample_gw_parameters(100000)

In [4]:
from ler.utils import save_json, load_json

# save_json('param_dict_bbh.json', param_dict)
param_dict = load_json('param_dict_bbh.json')
# convert to np.array
param_dict = {key: np.array(value) for key, value in param_dict.items()}

In [5]:
np.min(param_dict['mass_1']+param_dict['mass_2']), np.max(param_dict['mass_1']+param_dict['mass_2'])

(13.236215821976153, 1284.4836512781233)

## set-up bilby snr calculator

In [9]:
# Waveform arguments for waveform generation
output_jsonfile = 'inner_product_bilby.json'
waveform_approximant = 'IMRphenomD'
minimum_frequency = 20.0
sampling_frequency = 2048.0

waveform_arguments = dict(
    waveform_approximant=waveform_approximant,
    reference_frequency=20.0,
    minimum_frequency=minimum_frequency,
)

# psd objects
psds = dict()
psds["L1"] = "aLIGO_O4_high_asd.txt"
psds["H1"] = "aLIGO_O4_high_asd.txt"
psds["V1"] = "AdV_asd.txt"
list_of_detectors = ['L1', 'H1', 'V1']
ifos = bilby.gw.detector.InterferometerList(list_of_detectors)
psds_objects = []
for i, det in enumerate(list_of_detectors):
    psds_objects.append(
            bilby.gw.detector.PowerSpectralDensity(asd_file=psds[det])
        )

def bilby_inner_product(mass_1=10,
        mass_2=10,
        luminosity_distance=100.0,
        theta_jn=0.0,
        psi=0.0,
        phase=0.0,
        geocent_time=1246527224.169434,
        ra=0.0,
        dec=0.0,
        a_1=0.0,
        a_2=0.0,
        tilt_1=0.0,
        tilt_2=0.0,
        phi_12=0.0,
        phi_jl=0.0,
        lambda_1=0.0,
        lambda_2=0.0,
        eccentricity=0.0,
        gw_param_dict=False,
        mtot_min=9.96, 
        mtot_max=235.0,
        mtot_cut=True,
        fixed_duration=None,
        duration_min=None,
        duration_max=None,
    ):

    # if gw_param_dict is given, then use that
    if gw_param_dict is not False:
        mass_1, mass_2, luminosity_distance, theta_jn, psi, phase, geocent_time, ra, dec, a_1, a_2, tilt_1, tilt_2, phi_12, phi_jl, lambda_1, lambda_2, eccentricity  = get_gw_parameters(gw_param_dict)
    else:
        mass_1, mass_2, luminosity_distance, theta_jn, psi, phase, geocent_time, ra, dec, a_1, a_2, tilt_1, tilt_2, phi_12, phi_jl, lambda_1, lambda_2, eccentricity = get_gw_parameters(dict(mass_1=mass_1, mass_2=mass_2, luminosity_distance=luminosity_distance, theta_jn=theta_jn, psi=psi, phase=phase, geocent_time=geocent_time, ra=ra, dec=dec, a_1=a_1, a_2=a_2, tilt_1=tilt_1, tilt_2=tilt_2, phi_12=phi_12, phi_jl=phi_jl, lambda_1=lambda_1, lambda_2=lambda_2, eccentricity=eccentricity))

    if mtot_cut:
        mtot = mass_1 + mass_2
        n = np.where((mtot >= mtot_min) & (mtot <= mtot_max))[0]
    else:
        n = np.arange(len(mass_1))

    opt_snr_all = np.zeros((len(list_of_detectors), len(mass_1)))
    opt_snr_effective = np.zeros(len(mass_1))

    for idx in n:

        # IMPORTANT: time duration calculation for each of the mass combination
        if fixed_duration:
            duration = fixed_duration
        else:
            safety = 1.1
            approx_duration = safety * calculate_time_to_merger(mass_1[idx], mass_2[idx], minimum_frequency)
            duration = np.ceil(approx_duration + 2.0)

            if duration_max and (duration > duration_max):
                duration = duration_max  # IMRPheonomXPHM has maximum duration of 371s
            if duration_min and (duration < duration_min):
                duration = duration_min

        waveform_generator = bilby.gw.WaveformGenerator(
            duration=duration,
            sampling_frequency=sampling_frequency,
            frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
            waveform_arguments=waveform_arguments,
        )

        injection_parameters = dict(
            mass_1=mass_1[idx],
            mass_2=mass_2[idx],
            luminosity_distance=luminosity_distance[idx],
            geocent_time=geocent_time[idx],
            theta_jn=theta_jn[idx],
            ra=ra[idx],
            dec=dec[idx],
            psi=psi[idx],
            phase=phase[idx],
            a_1=a_1[idx],
            a_2=a_2[idx],
            tilt_1=tilt_1[idx],
            tilt_2=tilt_2[idx],
            phi_12=phi_12[idx],
            phi_jl=phi_jl[idx],
        )

        polas = waveform_generator.frequency_domain_strain(parameters=injection_parameters)

        opt_snr_dets = []
        snr_effective = 0.
        for j, det in enumerate(list_of_detectors):

            # need to compute the inner product for
            p_array = psds_objects[j].get_power_spectral_density_array(waveform_generator.frequency_array)

            idx2 = (p_array != 0.0) & (p_array != np.inf)
            hp_inner_hp = noise_weighted_inner_product(
                polas["plus"][idx2],
                polas["plus"][idx2],
                p_array[idx2],
                waveform_generator.duration,
            )
            hc_inner_hc = noise_weighted_inner_product(
                polas["cross"][idx2],
                polas["cross"][idx2],
                p_array[idx2],
                waveform_generator.duration,
            )

            Fp = ifos[j].antenna_response(
                ra=ra[idx],
                dec=dec[idx],
                time=geocent_time[idx],
                psi=psi[idx],
                mode='plus'
            )
            Fc = ifos[j].antenna_response(
                ra=ra[idx],
                dec=dec[idx],
                time=geocent_time[idx],
                psi=psi[idx],
                mode='cross'
            )
            print(f"Detector: {det}")
            print(f"hp_inner_hp: {hp_inner_hp}")
            print(f"hc_inner_hc: {hc_inner_hc}")
            print(f"Fp: {Fp}")
            print(f"Fc: {Fc}")

            snrs_sq = abs((Fp**2) * hp_inner_hp + (Fc**2) * hc_inner_hc)
            snr_effective += snrs_sq

            opt_snr_dets.append(np.sqrt(snrs_sq))

        opt_snr_all[:,idx] = opt_snr_dets
        opt_snr_effective[idx] = np.sqrt(snr_effective)

    # organizing the snr dictionary
    optimal_snr = dict()
    for j, det in enumerate(list_of_detectors):
        optimal_snr[det] = opt_snr_all[j]

    optimal_snr["optimal_snr_net"] = opt_snr_effective
    return optimal_snr

In [ ]:
bilby_inner_product(1,1,40, mtot_min=2)

In [ ]:
# measure the time taken for SNR calculation
start_time = datetime.now()
snr = bilby_inner_product(gw_param_dict=param_dict)
end_time = datetime.now()
execution_time_bilby = (end_time - start_time).total_seconds() * 1000  # convert to milliseconds
execution_time_bilby

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from gwsnr import GWSNR

gwsnr = GWSNR(npool=4, snr_type='inner_product', gwsnr_verbose=False, multiprocessing_verbose=False, mtot_cut=True)


Initializing GWSNR class...

psds not given. Choosing bilby's default psds




In [10]:
bilby_inner_product(30,29,500)

Detector: L1
hp_inner_hp: (5112.061451796086+0j)
hc_inner_hc: (5112.061451796086+0j)
Fp: 0.7391039324961559
Fc: -0.3263982271180008
Detector: H1
hp_inner_hp: (5112.061451796086+0j)
hc_inner_hc: (5112.061451796086+0j)
Fp: -0.42251192643910535
Fc: 0.2924673998038122
Detector: V1
hp_inner_hp: (3053.890391466239+0j)
hc_inner_hc: (3053.890391466239+0j)
Fp: -0.1213381362683638
Fc: 0.2944513812095745


{'L1': array([57.76856422]),
 'H1': array([36.74041044]),
 'V1': array([17.59941723]),
 'optimal_snr_net': array([70.6880772])}

In [2]:
gwsnr.compute_bilby_snr(mass_1=30.0, mass_2=29.0, luminosity_distance=500.0)

{'L1': array([57.76856371]),
 'H1': array([36.74040947]),
 'V1': array([17.59941695]),
 'optimal_snr_net': array([70.68807621])}

In [3]:
from gwsnr.numba.njit_functions import gps_to_gmst

In [4]:
gps_to_gmst(1246527224.169434)

44907.11975954579

In [13]:
np.searchsorted(np.array([1.,2.,3.]),0.1)-1, len(np.array([1.,2.,3.]))-1

(-1, 2)

In [ ]:
gwsnr.mtot_min, gwsnr.mtot_max

In [3]:
from ler.utils import load_json

param_dict = load_json('param_dict_bbh.json')
# convert to np.array
param_dict = {key: np.array(value) for key, value in param_dict.items()}

In [4]:
start_time = datetime.now()
snr = gwsnr.compute_bilby_snr(gw_param_dict=param_dict)
end_time = datetime.now()
execution_time_gwsnr = (end_time - start_time).total_seconds() * 1000  # convert to milliseconds
execution_time_gwsnr

14864.995

In [5]:
start_time = datetime.now()
snr = gwsnr.compute_bilby_snr(gw_param_dict=param_dict)
end_time = datetime.now()
execution_time_gwsnr = (end_time - start_time).total_seconds() * 1000  # convert to milliseconds
execution_time_gwsnr

16651.651

* 2-cores: 53.560 s
* 4-cores: 35.772 s

In [5]:
import numpy as np
65871/ np.array([16652, 94, 117, 187, 193])

array([  3.95574105, 700.75531915, 563.        , 352.2513369 ,
       341.30051813])

In [ ]:
gwsnr = GWSNR(
    snr_type='interpolation_aligned_spins',
    gwsnr_verbose=False,
)

In [ ]:
%timeit snr = gwsnr.snr_with_interpolation(gw_param_dict=param_dict)

In [ ]:
gwsnr = GWSNR(
    snr_type='interpolation_aligned_spins_jax',
    gwsnr_verbose=False,
)

In [ ]:
%timeit snr = gwsnr.snr_with_interpolation(gw_param_dict=param_dict)

In [ ]:
gwsnr = GWSNR(
    snr_type='interpolation_aligned_spins_mlx',
    gwsnr_verbose=False,
)

In [ ]:
%timeit snr = gwsnr.snr_with_interpolation(gw_param_dict=param_dict)

## jax-gpu
- 117 ms